# Project 3 - API and Web Data Scraping

Xabi López, April 10, 2021

## Importing libraries & creating functions 

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
def mes_num(x):
    meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    for i, j in zip(range(1,13),meses):
        if x.lower() == j:
            return i
        else:
            pass

In [4]:
def show_to_dict(show):
    artist = show.find('span', attrs={'itemprop':'name'}).text.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    year = int(show.find('td', attrs={'class':'fechaicono'}).text.split()[3])
    month = mes_num(show.find('td', attrs={'class':'fechaicono'}).text.split()[2])
    day = int(show.find('td', attrs={'class':'fechaicono'}).text.split()[1])
    location = show.find('td', attrs={'itemprop':'location'}).text.lstrip()
    return {'artist': artist, 'date':date(year, month, day).isoformat(),'location':location}

## Finding shows in the city & month we are interested in

In [5]:
city = input('City: ')

City: Burgos


In [6]:
month = input('Month: ')

Month: Septiembre


In [8]:
list_shows = []

url_city = f'https://www.laganzua.net/conciertos/{city.lower()}'
shows_city = requests.get(url_city)
soup = BeautifulSoup(shows_city.text, "html.parser")
try:
    pages = soup.findAll('ul', attrs={"class":"paginador"})[0]
    page_max = int([i.text for i in pages if i.text.isnumeric()][-1])
    for page in range(1,page_max+1):
        url_city = f'https://www.laganzua.net/conciertos/{city.lower()}-pagina-{page}'
        shows_city = requests.get(url_city)
        soup = BeautifulSoup(shows_city.text, "html.parser")
        shows = soup.findAll('tr', attrs={"class":"categoria"})
        for show in shows:
            list_shows.append(show_to_dict(show))
        shows = soup.findAll('tr', attrs={"class":"categoriab"})
        for show in shows:
            list_shows.append(show_to_dict(show))
except:
    url_city = f'https://www.laganzua.net/conciertos/{city.lower()}'
    shows_city = requests.get(url_city)
    soup = BeautifulSoup(shows_city.text, "html.parser")
    shows = soup.findAll('tr', attrs={"class":"categoria"})
    for show in shows:
        list_shows.append(show_to_dict(show))
    shows = soup.findAll('tr', attrs={"class":"categoriab"})
    for show in shows:
        list_shows.append(show_to_dict(show))

In [9]:
total_shows = pd.DataFrame(list_shows).sort_values('date').reset_index().drop('index',axis=1)
total_shows['date'] = pd.to_datetime(total_shows['date'])
total_shows['month']=total_shows['date'].dt.month
total_shows

,artist,date,location,month
0,Fernandocosta,2021-05-15,Sala Andén 56 Burgos,5
1,Sonorama Ribera 2021,2021-08-11,"Plaza Mayor Aranda Del Duero, Burgos",8
2,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,9
3,Anier,2021-11-06,Sala La Rúa Burgos,11


In [10]:
shows = total_shows[total_shows['month']==mes_num(month)]
shows[['artist','date','location']]

,artist,date,location
2,Texas,2021-09-26,Auditorio Fórum Evolución Burgos


## Obtaining all the songs from Spotify

In [11]:
client_id = '4a6d2893abff47e28c92847ffb2bbf54'
client_secret = '21542f29787e450590d3dd94584a275c'
sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [12]:
list_artists = []
for index, row in shows.iterrows():
    counter = 0
    try:
        max_songs = sp.search(q = row['artist'],  offset=0, limit=50)['tracks']['total']
        while counter < max_songs:
            list_songs = []
            artist_sp = sp.search(q = row['artist'],  offset=counter, limit=50)
            for song in artist_sp['tracks']['items']:
                artist_check = [i['name'] for i in song['artists']][0].replace('á','a').replace('é','e').replace('í','i').replace('ó','o')
                if artist_check.lower() == row['artist'].lower():
                    track = [row['artist'],row['date'],row['location'],song['name'],song['external_urls']['spotify']]
                    if track == []:
                        pass
                    else:
                        list_songs.append(track)
            if list_songs == []:
                pass
            else:
                list_artists.extend(list_songs)
            counter += 50
    except:
        pass

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.


In [13]:
tracks = pd.DataFrame(list_artists)
tracks.columns = ['artist', 'date', 'location', 'song', 'spotify']
tracks.dropna(inplace=True)
tracks.drop_duplicates(inplace=True)
tracks.groupby(['artist', 'date','location']).count()[['song']]

,,,song
artist,date,location,
Texas,2021-09-26,Auditorio Fórum Evolución Burgos,11


## Adding the lyrics

In [14]:
for index, row in tracks.iterrows():
    url_lyrics = f'https://api.lyrics.ovh/v1/{row["artist"]}/{row["song"]}'
    try:
        resp = requests.get(url_lyrics)
        response = resp.json()
        if [i for i in response.keys()][0]=='lyrics':
            tracks.loc[index,'lyrics'] = response['lyrics']
            tracks.loc[index,'url'] = resp.url
    except:
        pass

In [15]:
tracks.dropna(how = 'any', inplace=True)
tracks.to_csv(f'output/Conciertos_{city}_{month}.csv', header=True, index=False)

In [16]:
tracks

,artist,date,location,song,spotify,lyrics,url
0,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Say What You Want,https://open.spotify.com/track/0oSS8NUmfgaX52y...,Twenty seconds on the back time\r\nI feel you'...,https://api.lyrics.ovh/v1/Texas/Say%20What%20Y...
1,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Inner Smile,https://open.spotify.com/track/6i3kIF4sosXFhko...,"(one, two, three, four)\r\nYeh, yeh\r\nYeh, ye...",https://api.lyrics.ovh/v1/Texas/Inner%20Smile
2,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,I Don't Want A Lover,https://open.spotify.com/track/000u1dTg7y1XCDX...,I don't want a lover\r\nI just need a friend\r...,https://api.lyrics.ovh/v1/Texas/I%20Don't%20Wa...
3,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Hi,https://open.spotify.com/track/2uiapZlNT0NwJVc...,Hi hi hi hi high\r\nWe got\r\nHi hi hi hi high...,https://api.lyrics.ovh/v1/Texas/Hi
4,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Inner Smile,https://open.spotify.com/track/7zTbBG1yo6VUHbc...,"(one, two, three, four)\r\nYeh, yeh\r\nYeh, ye...",https://api.lyrics.ovh/v1/Texas/Inner%20Smile
5,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Summer Son,https://open.spotify.com/track/2ARTskci0SwksWt...,I'm tired of telling the story\r\nTired of tel...,https://api.lyrics.ovh/v1/Texas/Summer%20Son
6,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Black Eyed Boy,https://open.spotify.com/track/6fT9Oe3oy5gWvs8...,No I don't lack ambition\r\nCan't you see what...,https://api.lyrics.ovh/v1/Texas/Black%20Eyed%2...
9,Texas,2021-09-26,Auditorio Fórum Evolución Burgos,Hi,https://open.spotify.com/track/2j4SsFJ0HbVlT2b...,Hi hi hi hi high\r\nWe got\r\nHi hi hi hi high...,https://api.lyrics.ovh/v1/Texas/Hi


In [17]:
tracks.groupby(['artist', 'date','location']).count()[['song']]

,,,song
artist,date,location,
Texas,2021-09-26,Auditorio Fórum Evolución Burgos,8
